In [5]:
!pip install spotipy
!pip install pandas
!pip install scikit-learn
!pip install scikit-image
!pip install skimage
!pip install matplotlib
!pip install matplotlib.pyplot


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached skimage-0.0.tar.gz (757 bytes)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      
      *** Please install the `scikit-image` package (instead of `skimage`) ***
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Patricius Ronan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import sys
import re
import itertools
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
from collections import Counter
from skimage import io

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [ ]:
song_data = pd.read_csv('data-2.csv')
song_data.head(10)

In [ ]:
song_data['artists_song'] = song_data['artists_name'] +' ' + '-' + song_data['track_name']
song_data.tail() 

In [ ]:
genre_data = pd.read_csv('genres-2.csv')
genre_data.head()

In [ ]:
genre_data['genres'] = genre_data['genres'].apply(lambda x: [i for i in re.findall(r"'([^']*)'", x)])
genre_data.head()

In [ ]:
song_data.drop_duplicates('artists_song', inplace = True)
artists_explode = song_data[['artists_name','track_id']].explode('artists_name')

In [ ]:
artists_explode.head()

In [ ]:
#proses merge genre data dengan song data(artists_exploded)
artists_explode = artists_explode.merge(genre_data, how = 'left', left_on = 'artists_name',right_on = 'artists')
#menghilangkan NaN
artists_explode= artists_explode.dropna(subset=['genres'])

In [ ]:
artists_explode.head()

In [ ]:
artists_genres_combine = artists_explode.groupby('track_id')['genres'].apply(list).reset_index()
artists_genres_combine.head()

In [ ]:
artists_genres_combine['genres'] = artists_genres_combine['genres'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [ ]:
song_data = song_data.merge(artists_genres_combine[['track_id','genres']],on='track_id', how='left')

In [ ]:
song_data.head()

In [ ]:
float_column = song_data.dtypes[song_data.dtypes == 'float64'].index.values

In [ ]:
song_data['genres'] = song_data['genres'].apply(lambda y: y if isinstance(y,list) else [])

In [ ]:
onehotencoding_cols = 'popularity'

In [ ]:
song_data['popularity_rating'] = song_data['popularity'].apply(lambda x: int(x/5))

In [ ]:
song_data.head()

In [ ]:
def OneHotEncoding(data, column, new_name):
    TF = pd.get_dummies(data[column])
    feature_names = TF.columns
    TF.columns = [new_name + "||" + str(i) for i in feature_names]
    TF.reset_index(inplace=True, drop=True)
    return TF

In [ ]:
def create_feature_set(data, float_cols):

    TfIdf = TfidfVectorizer()
    TfIdf_matrix =  TfIdf.fit_transform(data['genres'].apply(lambda x: " ".join(x)))
    genre_data = pd.DataFrame(TfIdf_matrix.toarray())
    genre_data.columns = ['genre' + "|" + i for i in TfIdf.get_feature_names_out()]
    genre_data.reset_index(drop = True, inplace=True)

    floats = data[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
    OneHotEncoding_year = OneHotEncoding(data, 'year','year') * 0.5
    OneHotEncoding_pop = OneHotEncoding(data,'popularity_rating','pop') * 0.2

    final = pd.concat([genre_data, floats_scaled, OneHotEncoding_pop, OneHotEncoding_year], axis = 1)
     
    final['track_id']=data['track_id'].values
    
    return final

In [ ]:
complete_feature_set = create_feature_set(song_data, float_cols=float_column)

In [ ]:
pd.set_option('display.max_columns', None)
complete_feature_set.head()

In [ ]:
client_id = '71b8a60e3de244d6a14f3ddbb3532b4e'
client_secret= '78ad8c957e954325b07e3e37a8c139d5'
redirect_uri='http://localhost:5555/callback'

In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri=redirect_uri)

In [ ]:
scope = "playlist-read-private" 

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))

In [ ]:
name_and_id = {}
photo_list = {}

for i in sp.current_user_playlists()['items']:
    name_and_id[i['name']] = i['uri'].split(':')[2]
    photo_list[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
name_and_id

In [ ]:
#pengambilan lagu dalam playlist
def create_necessary_output(playlist_name,id_map,df):

    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for x, i in enumerate (sp.playlist(id_map[playlist_name])['tracks']['items']):

        playlist.loc[x, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[x, 'name'] = i['track']['name']
        playlist.loc[x, 'id'] = i['track']['id'] 
        playlist.loc[x, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[x, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    playlist = playlist[playlist['id'].isin(df['track_id'].values)]
    #playlist = playlist[(playlist['artist'].isin(df['artists_name'].values)) & (playlist['name'].isin(df['track_name'].values))].sort_values('date_added', ascending=True)

    return playlist

In [ ]:
playlist_90an = create_necessary_output("$$", name_and_id, song_data)

In [ ]:
playlist_90an

In [ ]:
def visualize_playlist(df):

    temp =  df['url'].values
    plt.figure(figsize=(15, int(math.ceil(0.7 * len(temp)))))
    column = 5

    for i, url in enumerate(temp): 
        plt.subplot(math.ceil(len(temp) / column), column, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color='w', fontsize=0.1)
        plt.yticks(color='w', fontsize=0.1)
        plt.xlabel(df['name'].values[i], fontsize=10)
        plt.tight_layout(h_pad=1, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()
    

In [ ]:
visualize_playlist(playlist_90an)

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['track_id'].isin(playlist_df['id'].values)]

    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['track_id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "track_id")

    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist_90an)

In [ ]:
def finalize_recommendation(df, features, nonplaylist_features):
 
    non_playlist_df = df[df['track_id'].isin(nonplaylist_features['track_id'].values)]
    non_playlist_df['cosine_sim'] = cosine_similarity(nonplaylist_features.drop('track_id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('cosine_sim',ascending = False).head(5)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
playlist_top40 = finalize_recommendation(song_data, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

In [ ]:
playlist_top40

In [ ]:
def visualize_recommendation(df):

    temp =  df['url'].values
    plt.figure(figsize=(15, int(math.ceil(0.7 * len(temp)))))
    column = 5

    for i, url in enumerate(temp): 
        plt.subplot(math.ceil(len(temp) / column), column, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color='w', fontsize=0.1)
        plt.yticks(color='w', fontsize=0.1)
        plt.xlabel(df['track_name'].values[i], fontsize=10)
        plt.tight_layout(h_pad=1, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()
    

In [ ]:
visualize_recommendation(playlist_top40)

In [ ]:
#Collaborative Filtering
#pengambilan lagu dalam playlist
def create_collaborative_output(playlist_name,id_map):

    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for x, i in enumerate (sp.playlist(id_map[playlist_name])['tracks']['items']):

        playlist.loc[x, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[x, 'name'] = i['track']['name']
        playlist.loc[x, 'id'] = i['track']['id'] 
        playlist.loc[x, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[x, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  

    return playlist

In [ ]:
playlists_to_compare = [name for name in name_and_id.keys() if name != "!"]

In [ ]:
songs_data_playlist_to_compare = create_collaborative_output("!", name_and_id)[['id', 'name', 'artist', 'url']]

In [ ]:
songs_data_playlist_to_compare

In [ ]:
songs_data_other_playlists = []

for playlist_name in playlists_to_compare:
    if playlist_name != "!":
        playlist_data = create_collaborative_output(playlist_name, name_and_id)[['id', 'name', 'artist','url']]
        songs_data_other_playlists.append(playlist_data)

songs_data_other_playlists = pd.concat(songs_data_other_playlists, ignore_index=True)

In [ ]:
songs_data_other_playlists

In [ ]:
#Mencari common songs
common_songs = []

song_ids_input = set(songs_data_playlist_to_compare['id'])

playlist_names = [name for name in name_and_id.keys() if name != "!"]

for playlist_name in playlist_names:
    if playlist_name != "!":
        playlist_data = create_collaborative_output(playlist_name, name_and_id)
        playlist_id = name_and_id[playlist_name] 
        for _, row in playlist_data.iterrows():
            song_id = row['id']
            if song_id in song_ids_input:
                
                playlist_id = name_and_id[playlist_name]
                common_songs.append(f"{row['id']} - {row['name']} - {row['artist']} - {row['url']} (from playlist : {playlist_id} - {playlist_name})")


In [ ]:
common_songs_id = set([playlist_id.split("-")[0].strip() for playlist_id in common_songs])

In [ ]:
playlist_ids = set([playlist_id.split("from playlist :")[1].strip() for playlist_id in common_songs])

In [ ]:
playlist_ids_final = set([playlist_ids.split(" - ")[0] for playlist_ids in playlist_ids])

In [ ]:
recommended_songs = []
recommended_songs_results = []
for playlist_id in playlist_ids_final:
    playlist_name = None
    for name, id in name_and_id.items():
        if id == playlist_id:
            playlist_name = name
            break

    if playlist_name is not None:
        playlist_data = create_collaborative_output(playlist_name, name_and_id)
        
        for z, row in playlist_data.iterrows():
            song_id = row['id']

            if song_id not in common_songs_id:
                recommended_songs.append(song_id)

id_frequencies = Counter(recommended_songs)

sorted_song_frequencies = id_frequencies.most_common()

for song_id, frequency in sorted_song_frequencies:
    song_datas = songs_data_other_playlists[songs_data_other_playlists['id'] == song_id]

    if frequency > 1:
        recommended_songs.append(song_id)
        artist = song_datas['artist'].values[0]
        name = song_datas['name'].values[0]
        url = song_datas['url'].values[0]
        recommended_songs_results.append({'ID Lagu': song_id, 'Artis': artist, 'Nama Lagu': name, 'url' : url, 'Duplikat': frequency})
    
    top_5_collaboration = recommended_songs_results[:5]

    top_5_collaboration_df = pd.DataFrame(top_5_collaboration)

In [ ]:
def visualize_recommendation_collaborative(df):

    temp =  df['url'].values
    plt.figure(figsize=(15, int(math.ceil(0.7 * len(temp)))))
    column = 5

    for i, url in enumerate(temp): 
        plt.subplot(math.ceil(len(temp) / column), column, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color='w', fontsize=0.1)
        plt.yticks(color='w', fontsize=0.1)
        plt.xlabel(df['Nama Lagu'].values[i], fontsize=10)
        plt.tight_layout(h_pad=1, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()
    

In [ ]:
visualize_recommendation_collaborative(top_5_collaboration_df)

In [ ]:
#Rekomendasi tanpa Algoritma
random_without_algorithm = random.sample(range(len(song_data)), 5)
random_without_algorithms = song_data.iloc[random_without_algorithm]
random_without_algorithm_5 = random_without_algorithms[['track_id', 'track_name', 'artists_name']]

In [ ]:
random_without_algorithm_5